In [1]:
%load_ext lab_black
%cd ..

/home/shim/cev/dl/log-analytics


In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig, DistilBertTokenizer
from torch.optim import AdamW
import torch.nn.functional as F
from tqdm import tqdm
import sys
import re
from collections import defaultdict

In [387]:
import logging

logging.getLogger("pytorch_transformers.tokenization_utils").setLevel(logging.ERROR)

In [8]:
seasons = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

In [3]:
train = pd.read_csv("data/ori/train.csv")

In [4]:
train

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...
...,...,...,...
472967,472967,0,Feb 28 10:10:06 localhost logstash: 7738 error:
472968,472968,1,type=SYSCALL msg=audit(1611890993.458:321827):...
472969,472969,0,"Oct 12 02:20:29 localhost kibana: {""type"":""log..."
472970,472970,0,"Jan 15 09:02:43 localhost kibana: {""type"":""err..."


In [134]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [355]:
def text_tokenizing(text):
    text = text.replace("\n", " { ")
    text = text.replace("\\n", " { ")
    text = re.sub(r"[\[\]\(\)\"'\s\|\+\*\&\^\%\$\#\@\!\~\`;\?\<\>]+", " ", text)
    # text = text.replace("=", " = ")
    text = text.replace("{", "[SEP]")
    text = text.replace("}", "[SEP]")
    # text = text.replace(".", ". ")

    text = text.split()
    while True:
        if text[0].isdigit() or text[0].lower() in seasons:
            text = text[1:]
        else:
            break

    return text

In [356]:
def myfilter(x):
    if x.lower() in seasons:
        return ""

    # if x.isdigit():
    #     return ""

    if re.fullmatch(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", x):
        return "IP"

    return x

In [392]:
j = 4
print(train.full_log[j])

type=SYSCALL msg=audit(1603094402.016:52981): arch=c000003e syscall=2 success=yes exit=3 a0=7ff220e805a4 a1=80000 a2=1 a3=7ff2210864f8 items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=45 comm="date" exe="/usr/bin/date" subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key="audit-wazuh-r" type=CWD msg=audit(1603094402.016:52981):  cwd="/root" type=PATH msg=audit(1603094402.016:52981): item=0 name="/etc/ld.so.cache" inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603094402.016:52981): proctitle=64617465002B2564


In [394]:
out = " ".join(list(filter(None, map(myfilter, text_tokenizing(train.full_log[j])))))
out

'type=SYSCALL msg=audit 1603094402.016:52981 : arch=c000003e syscall=2 success=yes exit=3 a0=7ff220e805a4 a1=80000 a2=1 a3=7ff2210864f8 items=1 ppid=5877 pid=5878 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty= none ses=45 comm= date exe= /usr/bin/date subj=system_u:system_r:sysstat_t:s0-s0:c0.c1023 key= audit-wazuh-r type=CWD msg=audit 1603094402.016:52981 : cwd= /root type=PATH msg=audit 1603094402.016:52981 : item=0 name= /etc/ld.so.cache inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit 1603094402.016:52981 : proctitle=64617465002B2564'

In [395]:
len(out.split())

60

In [396]:
tokens = tokenizer.encode(out)

In [397]:
print(len(tokens))
tokens = tokens[:512]

352


In [398]:
tokenizer.decode(tokens)

'type = syscall msg = audit 1603094402. 016 : 52981 : arch = c000003e syscall = 2 success = yes exit = 3 a0 = 7ff220e805a4 a1 = 80000 a2 = 1 a3 = 7ff2210864f8 items = 1 ppid = 5877 pid = 5878 auid = 0 uid = 0 gid = 0 euid = 0 suid = 0 fsuid = 0 egid = 0 sgid = 0 fsgid = 0 tty = none ses = 45 comm = date exe = / usr / bin / date subj = system _ u : system _ r : sysstat _ t : s0 - s0 : c0. c1023 key = audit - wazuh - r type = cwd msg = audit 1603094402. 016 : 52981 : cwd = / root type = path msg = audit 1603094402. 016 : 52981 : item = 0 name = / etc / ld. so. cache inode = 69008420 dev = fd : 00 mode = 0100644 ouid = 0 ogid = 0 rdev = 00 : 00 obj = unconfined _ u : object _ r : ld _ so _ cache _ t : s0 objtype = normal type = proctitle msg = audit 1603094402. 016 : 52981 : proctitle = 64617465002b2564'

---

In [294]:
train.level[train.level == 2]

39057     2
87600     2
109510    2
262892    2
294072    2
294127    2
343011    2
345903    2
365348    2
376681    2
380583    2
400735    2
Name: level, dtype: int64

In [295]:
train.full_log[39057]

'The average number of logs between 10:00 and 11:00 is 5399. We reached 37090.'

In [296]:
for i in range(7):
    print(i, len(train.level[train.level == i]))

0 334065
1 132517
2 12
3 4141
4 10
5 2219
6 8


In [301]:
for i, full_log in train.full_log[train.level == 2].iteritems():
    print(i, full_log)
    print()

39057 The average number of logs between 10:00 and 11:00 is 5399. We reached 37090.

87600 The average number of logs between 5:00 and 6:00 is 222. We reached 77946.

109510 The average number of logs between 6:00 and 7:00 is 6420. We reached 28494.

262892 The average number of logs between 21:00 and 22:00 is 5325. We reached 95.

294072 The average number of logs between 5:00 and 6:00 is 8362. We reached 93476.

294127 The average number of logs between 4:00 and 5:00 is 2806. We reached 11476.

343011 The average number of logs between 14:00 and 15:00 is 9734. We reached 26958.

345903 The average number of logs between 15:00 and 16:00 is 4411. We reached 9487.

365348 Sep 18 11:02:59 localhost sudo:  apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/curl -XGET localhost:9200/_cat/snapshots/esild_backup

376681 The average number of logs between 2:00 and 3:00 is 9428. We reached 75572.

380583 The average number of logs between 10:00 and 11:00 is 12612. W

In [308]:
for i, full_log in train.full_log[train.level == 4].iteritems():
    print(i, full_log)
    print()

64919 Nov 29 05:44:21 localhost sshd[6008]: Did not receive identification string from 211.253.243.66 port 57487

85354 Nov 29 22:16:32 sv260 sshd[39585]: Did not receive identification string from 211.253.243.66

123069 Nov 3 10:38:05 localhost sshd[76373]: Did not receive identification string from 192.168.0.195 port 3819

167164 Nov 21 01:02:56 sv260 sshd[9196]: Did not receive identification string from 211.253.243.66

184483 Nov 27 08:32:12 sv260 sshd[3110]: Did not receive identification string from 192.168.0.195

230131 Nov 19 01:03:09 sv260 sshd[13841]: Did not receive identification string from 192.168.0.195

321526 Nov 12 04:27:24 localhost sshd[98418]: Did not receive identification string from 61.41.101.142 port 8975

348055 Jan 27 20:20:54 localhost sshd[5025]: Did not receive identification string from 35.184.222.44 port 46232

348684 Nov 3 06:03:17 localhost sshd[4346]: Did not receive identification string from 192.168.0.181 port 82066

369842 Nov 8 20:25:42 sv260 sshd[

In [309]:
for i, full_log in train.full_log[train.level == 6].iteritems():
    print(i, full_log)
    print()

87783 Jan 30 08:23:17 localhost sshd[18415]: Bad protocol version identification '\003' from 78.128.113.18 port 1073

170386 Feb  8 16:16:47 localhost kernel: device virbr0-nic entered promiscuous mode

213304 Mar  8 15:29:30 localhost kernel: device enp2s0 entered promiscuous mode

221847 Jan 29 11:28:59 localhost useradd[88679]: new group: name=test, GID=1001

223644 Jan 22 15:18:49 localhost sshd[19015]: error: maximum authentication attempts exceeded for root from 192.168.0.197 port 61153 ssh2 [preauth]

361823 Feb  2 17:22:58 localhost kernel: device virbr0-nic entered promiscuous mode

406796 Feb  4 09:59:33 localhost kernel: device virbr0-nic entered promiscuous mode

429489 Feb  4 09:59:43 localhost kernel: device enp2s0 entered promiscuous mode



In [345]:
for i, full_log in list(train.full_log[train.level == 5].iteritems())[100:110]:
    print(i, full_log)
    print()

22650 oscap: msg: "xccdf-result", scan-id: "0001603069960", content: "ssg-centos-7-ds.xml", title: "Enable GNOME3 Screensaver Idle Activation", id: "xccdf_org.ssgproject.content_rule_dconf_gnome_screensaver_idle_activation_enabled", result: "fail", severity: "medium", description: "To activate the screensaver in the GNOME3 desktop after a period of inactivity, add or set idle-activation-enabled to true in /etc/dconf/db/local.d/00-security-settings. For example: [org/gnome/desktop/screensaver] idle_activation_enabled=true Once the setting has been added, add a lock to /etc/dconf/db/local.d/locks/00-security-settings-lock to prevent user modification. For example: /org/gnome/desktop/screensaver/idle-activation-enabled After the settings have been set, run dconf update.", rationale: "A session time-out lock is a temporary action taken when a user stops work and moves away from the immediate physical vicinity of the information system but does not logout because of the temporary nature of 